In [8]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, model_phonebook, run_mcmc
from codebase.plot import get_post_df, plot_density
from codebase.run_tlk import model_phonebook
from tqdm import tqdm
from scipy.special import logsumexp
import altair as alt
alt.data_transformers.disable_max_rows()



import pystan

In [15]:
log_dir = 'log/fabian_sim/EZ/20210917_221719_ezsim_1factor_m9/'
data = load_obj('data', log_dir)


## Run MCMC

In [3]:

model_num = 9

sm = load_obj('sm', 'log/compiled_models/model'+str(model_num)+'/')

param_names = model_phonebook(model_num)['param_names']
latent_names = model_phonebook(model_num)['latent_names']
stan_names = model_phonebook(model_num)['stan_names']

fit_run = run_mcmc(
    data.get_stan_data(),
    sm,
    num_samples = 1000,
    num_warmup = 1000,
    adapt_engaged=True,
    num_chains = 1,
    log_dir = './log/20210725_224158_debug/'
)
ps = fit_run.extract(permuted=False, pars=param_names)


## Run IBIS

In [12]:
# load existing results

ibis = load_obj('particles', log_dir)


In [4]:
gen_model = 0
model_num = 1


for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(ibis.particles[name])
    w = exp_and_normalise(ibis.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))



Estimate
[-0.09  0.02  0.04 -0.37 -0.32 -1.95]


Estimate
[[ 1.58  0.17  0.38  0.29  0.33  0.16]
 [ 0.17  1.73  0.35  0.36  0.43  0.12]
 [ 0.38  0.35  2.12  0.68  0.76  0.28]
 [ 0.29  0.36  0.68  1.54  1.12  0.47]
 [ 0.33  0.43  0.76  1.12  1.86  0.51]
 [ 0.16  0.12  0.28  0.47  0.51 10.6 ]]


## Plot Both results

In [16]:
# have to resample particles to get rid of weights
ibis.resample_particles()
particles = ibis.particles


In [17]:
particles['beta'].shape

(500, 6, 2)

In [18]:
param = 'beta'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'mcmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))

NameError: name 'ps' is not defined

In [8]:
param = 'alpha'
df = get_post_df(np.squeeze(ps[param]))
df['source'] = 'hmc'
df2 = get_post_df(particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))


alt.Chart(...)